In [1]:
import sys
sys.path.append('/Users/victoria/Documents/Scripts/Python/DGL-PTM/dgl_ptm')
import dgl_ptm
import os
os.environ["DGLBACKEND"] = "pytorch"

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
model = dgl_ptm.PovertyTrapModel(model_identifier='testwithnn')

In [4]:
import torch
model.set_model_parameters(**{'number_agents': 100 , 
    'seed':1,
    'gamma_vals':torch.tensor([0.3,0.45]) , #for pseudo income
    'sigma_dist': {'type':'uniform','parameters':[0.05,1.94],'round':True,'decimals':1},
    'cost_vals': torch.tensor([0.,0.45]), #for pseudo income
    'a_theta_dist': {'type':'uniform','parameters':[0.1,1],'round':False,'decimals':None},
    'sensitivity_dist':{'type':'uniform','parameters':[0.0,1],'round':False,'decimals':None},
    'capital_dist': {'type':'uniform','parameters':[0.1,10.],'round':False,'decimals':None}, 
    'alpha_dist': {'type':'normal','parameters':[1.08,0.074],'round':False,'decimals':None},
    'lambda_dist': {'type':'uniform','parameters':[0.05,0.94],'round':True,'decimals':1},
    'initial_graph_type': 'barabasi-albert',
    'initial_graph_args': {'seed': 1, 'new_node_edges':1},
    'device': 'cpu',
    'step_target':20,
    'steering_parameters':{'npath':'./agent_data.zarr',
                            'epath':'./edge_data', 
                            'ndata':['all_except',['a_table']],
                            'edata':['all'],
                            'mode':'w',
                            'wealth_method':'weighted_transfer',
                            'income_method':'income_generation',
                            'tech_gamma': torch.tensor([0.3,0.35,0.45]),
                            'tech_cost': torch.tensor([0,0.15,0.65]),
                            'consume_method':'past_shock_bellman_consumption',
                            'nn_path': "/nn_data/both_PudgeSixLayer_1024/0723_110813/model_best.pth",
                            'adapt_m':torch.tensor([0,0.5,0.9]),
                            'adapt_cost':torch.tensor([0,0.25,0.45]),
                            'depreciation': 0.6,
                            'discount': 0.95,
                            'm_theta_dist': {'type':'beta','parameters':[4.13,0.07],'round':False,'decimals':None},
                            'del_method':'size',
                            'noise_ratio': 0.05,
                            'local_ratio': 0.25,
                            'homophily_parameter':0.69,
                            'characteristic_distance':35, 
                            'truncation_weight':1.0e-10,
                            'step_type':'custom'}})

print(model.config.number_agents)  
print(model.config.steering_parameters)                        

100
edata=['all'] epath='testwithnn/edge_data' format='xarray' mode='w' ndata=['all_except', ['a_table']] npath='testwithnn/agent_data.zarr' wealth_method='weighted_transfer' income_method='income_generation' consume_method='past_shock_bellman_consumption' nn_path='/nn_data/both_PudgeSixLayer_1024/0723_110813/model_best.pth' capital_update_method='default' characteristic_distance=35 homophily_parameter=0.69 adapt_m=tensor([0.0000, 0.5000, 0.9000]) adapt_cost=tensor([0.0000, 0.2500, 0.4500]) depreciation=0.6 discount=0.95 m_theta_dist=MThetaDist(type='beta', parameters=tensor([4.1300, 0.0700]), round=False, decimals=None) tech_gamma=tensor([0.3000, 0.3500, 0.4500]) tech_cost=tensor([0.0000, 0.1500, 0.6500]) del_method='size' del_threshold=0.05 noise_ratio=0.05 local_ratio=0.25 truncation_weight=1e-10 step_type='custom' data_collection_period=1 data_collection_list=None


In [5]:
model.initialize_model()
print (model.steering_parameters['modelTheta'] )

using seed 1 for network creation.
1 1
tensor([1.0000, 1.0000, 1.0000, 0.9487, 0.9421, 0.9997, 1.0000, 0.9990, 1.0000,
        1.0000, 0.9999, 0.9811, 1.0000, 1.0000, 0.8913, 1.0000, 0.9454, 1.0000,
        0.9883, 1.0000])


In [6]:
model.run()

performing step 0 of 20
Agents in equation violation: 8
Agents in personal/actual violation: 4
...because of i_a: 0
performing step 1 of 20
Agents in equation violation: 3
Agents in personal/actual violation: 3
...because of i_a: 0
performing step 2 of 20
Agents in equation violation: 4
Agents in personal/actual violation: 4
...because of i_a: 0
performing step 3 of 20
Agents in equation violation: 5
Agents in personal/actual violation: 5
...because of i_a: 0
performing step 4 of 20
Agents in equation violation: 9
Agents in personal/actual violation: 9
...because of i_a: 0
performing step 5 of 20
Agents in equation violation: 14
Agents in personal/actual violation: 14
...because of i_a: 0
performing step 6 of 20
Agents in equation violation: 19
Agents in personal/actual violation: 19
...because of i_a: 0
performing step 7 of 20
Agents in equation violation: 24
Agents in personal/actual violation: 24
...because of i_a: 0
performing step 8 of 20
Agents in equation violation: 26
Agents in